In [1]:
import pandas as pd
from datetime import datetime
import re


In [2]:
rawpath = "../raw"

df = pd.read_excel(f"{rawpath}/Gesamt_Kbl.xlsx", names=['signatur','urheber','idn','titel','zusatz','bem2','ort','verlag','entstehung','blatt','bild','reihe','preis','schlagwort','bemerkung','technik','motiv', 'idn_technik', 'mappe', 'grafiker'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df = df[(df.signatur.str.startswith('Kbl'))]
df = df.drop_duplicates(subset=['signatur'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11749 entries, 0 to 14047
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   signatur     11749 non-null  string
 1   urheber      10672 non-null  string
 2   idn          8364 non-null   string
 3   titel        11746 non-null  string
 4   zusatz       1639 non-null   string
 5   bem2         11673 non-null  string
 6   ort          11702 non-null  string
 7   verlag       11697 non-null  string
 8   entstehung   11244 non-null  string
 9   blatt        11715 non-null  string
 10  bild         9573 non-null   string
 11  reihe        2078 non-null   string
 12  preis        10709 non-null  string
 13  schlagwort   11375 non-null  string
 14  bemerkung    3962 non-null   string
 15  technik      11662 non-null  string
 16  motiv        6415 non-null   string
 17  idn_technik  11643 non-null  string
 18  mappe        413 non-null    string
 19  grafiker     278 non-null

In [3]:
# überspringe Zeilen mit mappe ja
# if pd.isna(row.mappe):
# df2=df.dropna(subset=['mappe'])
# df2.info()

In [4]:
regsuche = re.compile(r"\[?[Hhrsg]{4}\.?\:?\]?(\s?[Vvonm]{1,3}\.?)?")
df['hrsg'] = df['zusatz'].str.split(regsuche, regex=True, n=1, expand=True)[2].str.strip()

In [5]:
df['hrsg2'] = df['hrsg'].str.split(',', expand=True)[1].str.strip()

regsuche =re.compile(r"[Dt\.\sZentralinst]{16}\s[Ff]{1}[\.\sLehrmittel]{12}")
df['hrsg2'] = df['hrsg2'].str.replace(regsuche, '!004686209!', regex=True)


In [6]:
regsuche =re.compile(r"[Dt\.\sPpädgog\.\sZentralinst\.]{24,}")
df['hrsg'] = df['hrsg'].str.replace(regsuche, '!004686144!', regex=True)



In [7]:
def val_idn(idn):
    if len(idn) < 9:
        return (9 - len(idn)) * '0' + idn
    else:
        return idn

In [8]:
# Signatur, Geistiger schöpfer (Nachname, Vorname), IDN, TItel, Zusatztitel,, 4201, Ort, Verlag, Jahr, Blattmaß, Bildmaß, Reihenzugehörigkeit, Preis, Schlagwort, Bemerkung, Technik, IDN motiv, IDN Technik, hrsg, hrsg2

#Sigantur = 7100 und 7109 

In [36]:
from threading import local


def pica_schreiben(row):
 
    pica = """0500 Pa
0501 unbewegtes Bild$bsti
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
#4stellig suchen für 1100, 1110 der ganze Text der row.entstehung, falls nichts in row.entstehung steht dann folgender text 1100 1990$n[nicht nach 1990] re.search r"1\d{3}"
    if  (pd.notna(row.entstehung)) and (year := re.findall(r"\d{4}", str(row.entstehung))):
        pica += f"1100 {year[0]}\n1110 {row.entstehung}$4ezth\n"
    else:
        pica += f"1100 1991$n[nicht nach 1991]\n"

    pica +=f"1131 !041512367!\n"
    if 'year' in locals():
        if int(year[0]) > 1944:
            pica +=f"1700 /1XA-DDDE\n"
        elif int(year[0]) < 1945:
            pica +=f"1700 /1XA-DXDE\n"

    pica +=f"2105 94,P01\n"

    if pd.notna(row.preis):
        pica +=f"2000 {row.preis}\n"

    if pd.notna(row.idn):
        pica +=f"3010 !{val_idn(row.idn)}!$BKünstler$4art\n" 
    elif pd.notna(row.urheber):
        pica +=f"3010 {row.urheber}$BKünstler$4art\n"

    if pd.notna(row.grafiker):
        pica += f"3010 {row.grafiker.split(' ')[1].strip()}, {row.grafiker.split(' ')[0].strip()}$BKünstler$4art\n"

    if pd.notna(row.hrsg):
        pica +=f"3100 {row.hrsg}$BHerausgebendes Organ$4isb\n"
    if pd.notna(row.hrsg2):
        pica +=f"3110 {row.hrsg2}$BHerausgebendes Organ$4isb\n"

    if pd.notna(row.titel) and pd.notna(row.zusatz) and pd.notna(row.urheber) and pd.notna(row.grafiker):
        pica +=f"4000 {row.titel} : {row.zusatz} / {row.urheber.split(',')[1].strip()} {row.urheber.split(',')[0].strip()}. Grafiker: {row.grafiker}\n"
    elif pd.notna(row.titel) and pd.isna(row.zusatz) and pd.notna(row.urheber):
        pica +=f"4000 {row.titel} / {row.urheber.split(',')[1].strip()} {row.urheber.split(',')[0].strip()}\n"
    elif pd.notna(row.titel) and pd.isna(row.zusatz) and pd.isna(row.urheber):
        pica +=f"4000 {row.titel}\n"

    if pd.notna(row.ort) and pd.notna(row.verlag):
        pica +=f"4030 {row.ort} : {row.verlag}\n"
    elif pd.isna(row.ort) and pd.notna(row.verlag):
        pica +=f"4030 [Ort nicht ermittelbar] : {row.verlag}\n"
    elif pd.notna(row.ort) and pd.isna(row.verlag):
        pica +=f"4030 {row.ort} : [Verlag nicht ermittelbar]\n"
    elif pd.isna(row.ort) and pd.isna(row.verlag):
        pica +=f"4030 [Ort nicht ermittelbar] : [Verlag nicht ermittelbar]\n"

    pica +=f"4060 1 ungezähltes Blatt\n"

    if blattwerte := re.findall(r"(\d+)\s?x\s?(\d+)", str(row.blatt)):
        pica += f"4062 {blattwerte[0][0]} x {blattwerte[0][1]} mm$b{blattwerte[0][0]}$h{blattwerte[0][1]}$4bltt\n"

    if bildwerte := re.findall(r"(\d+)\s?x\s?(\d+)", str(row.bild)):
        pica += f"4062 {bildwerte[0][0]} x {bildwerte[0][1]} mm$b{bildwerte[0][0]}$h{bildwerte[0][1]}$4bild\n"

    pica +=f"4105 !!\n"

    if pd.notna(row.bem2) and pd.notna(row.bemerkung):
        pica +=f"4201 {row.bem2}. {row.bemerkung}\n"

    elif pd.isna(row.bem2) and pd.notna(row.bemerkung):
        pica +=f"4201 {row.bemerkung}\n"

    elif pd.notna(row.bem2) and pd.isna(row.bemerkung):
        pica +=f"4201 {row.bem2}\n"

    if pd.notna(row.reihe):
        pica +=f"4190 {row.reihe}\n"

    if pd.notna(row.motiv):
        pica +=f"5100 {row.motiv}\n"

    if pd.notna(row.idn_technik):
        pica +=f"5590 [Technik]\n5590 !{val_idn(row.idn_technik)}!"


        



    pica += "\t\n"
    return pica

In [37]:
for index, row in df[pd.notna(df.hrsg2)].sample(10).iterrows():
    print(pica_schreiben(row))

0500 Pa
0501 unbewegtes Bild$bsti
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1963
1110 1963$4ezth
1131 !041512367!
1700 /1XA-DDDE
2105 94,P01
3010 !174262183!$BKünstler$4art
3100 !004686144!, Dt. Zentralinst. f. Lehrmittel$BHerausgebendes Organ$4isb
3110 !004686209!$BHerausgebendes Organ$4isb
4030 Berlin : Volk u. Wissen
4060 1 ungezähltes Blatt
4062 865 x 1220 mm$b865$h1220$4bltt
4062 635 x 1070 mm$b635$h1070$4bild
4105 !!
4201 Nach e. Gemälde, Mehrfarb. Offsetdruck. Best.Nr 71129-1, Preis nach Anfrage; Versorgung d. Schulen u. Berufsschulen durch d. Verlag zu Schulabgabepreisen
5100 !041569717!
5590 [Technik]
5590 !041333357!	

0500 Pa
0501 unbewegtes Bild$bsti
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1963
1110 1963$4ezth
1131 !041512367!
1700 /1XA-DDDE
2105 94,P01
3100 !004686144!, Dt. Zentralinst. F. Lehrmittel$BHerausgebendes Organ$4isb
3110 !004686209!$BHerausgebendes Organ$4isb
4030 Berlin : Volk und Wissen
4060 1 ungezähltes Blatt
4